In [2]:
%%latex
Tri-level MPC
\begin{align}
\max_{\mathbf{d,r,\hat{n}}} \quad &\sum_{t\in\mathcal{T}}\sum_{(i,j) \in \mathcal{E}}d_{ijt}p_{ijt} 
- \beta\sum_{t\in\mathcal{T}}\sum_{(i,j) \in \mathcal{E}}(r_{ijt}+d_{ijt})\tau_{ij} \\
\rm{s.t.}\quad & n_{i,t+1} = n_{it} - \sum_{j: (i,j)\in\mathcal{E}}(d_{ijt}+r_{ijt}) 
    + \sum_{j: (j,i)\in\mathcal{E}}(d_{ij,t-\tau_{ij}}+r_{ij,t-\tau_{ij}}) \\
    & \sum_{j:(i,j)\in\mathcal{E}}(d_{ijt}+r_{ijt}) \leq n_{it} \\
    & \mathbf{d}_t \in F(\mathbf{n}_t) \\
    & \mathbf{r}_t \in G(\mathbf{n}_t, \mathbf{\hat{n}}_t) \\
    & \hat{n} \in \mathbb{Z}_+
\end{align} 

$F(\cdot)$ is the set of optimal solution of the matching problem
\begin{align}
\max_{\mathbf{d}} \quad & \sum_{(i,j)\in\mathcal{E}}d_{ijt}p_{ijt} \\
\quad & \sum_{j:(i,j) in \mathcal{E}}d_{ijt} \leq n_{it}\\
& d_{ijt} \leq \lambda_{ijt} 
\end{align}

$G()$ is the set of optimal solution of the rebalancing problem 
\begin{align}
\min_{\mathbf{r}} \quad & \sum_{(i,j)\in\mathcal{E}}r_{ijt}\tau_{ij} \\
\quad & \sum_{j:(j,i)\in\mathcal{E}}r_{ij} - \sum_{j:(i,j)\in\mathcal{E}}r_{ij} \geq \hat{n}_{it} - n_{it}
& \sum_{j:(i,j) in \mathcal{E}}r_{ijt} \leq n_{it}\\
\end{align}

One alternative for G is the set of optimal solution of the rebalancing problem
\begin{align}
\min_{\mathbf{r}} \quad & \sum_{(i,j)\in\mathcal{E}}r_{ijt}\tau_{ij} \\
\quad & \sum_{j:(j,i)\in\mathcal{E}}r_{ij} = a_{it} \\
& \sum_{j:(i,j)\in\mathcal{E}}r_{ij} = b_{it}\\
\end{align}

<IPython.core.display.Latex object>

In [3]:
from src.envs.env_two_step import Scenario, AMoD, Star2Complete
from src.misc.utils import mat2str, dictsum
from src.algos.MPC import MPC
import os
import subprocess
from collections import defaultdict
import numpy as np
platform = 'linux'
if platform == 'windows':
    CPLEXPATH = "C:/Program Files/ibm/ILOG/CPLEX_Studio1210/opl/bin/x64_win64/"
elif platform == 'mac':
    CPLEXPATH = "/Applications/CPLEX_Studio1210/opl/bin/x86-64_osx/"
elif platform == 'linux':
    CPLEXPATH = "/opt/ibm/ILOG/CPLEX_Studio128/opl/bin/x86-64_linux/"
    
def solveRebFlow(env,res_path,desiredAcc,CPLEXPATH):
    t = env.time
    accRLTuple = [(n,int(round(desiredAcc[n]))) for n in desiredAcc]
    accTuple = [(n,int(env.acc[n][t+1])) for n in env.acc]
    edgeAttr = [(i,j,env.G.edges[i,j]['time']) for i,j in env.G.edges]
    modPath = os.getcwd().replace('\\','/')+'/mod/'
    OPTPath = os.getcwd().replace('\\','/')+'/MPC/'+res_path
    if not os.path.exists(OPTPath):
        os.makedirs(OPTPath)
    datafile = OPTPath + f'data_{t}.dat'
    resfile = OPTPath + f'res_{t}.dat'
    with open(datafile,'w') as file:
        file.write('path="'+resfile+'";\r\n')
        file.write('edgeAttr='+mat2str(edgeAttr)+';\r\n')
        file.write('accInitTuple='+mat2str(accTuple)+';\r\n')
        file.write('accRLTuple='+mat2str(accRLTuple)+';\r\n')
    modfile = modPath+'minRebDistRebOnly.mod'
    if CPLEXPATH is None:
        CPLEXPATH = "/opt/ibm/ILOG/CPLEX_Studio128/opl/bin/x86-64_linux/"
    my_env = os.environ.copy()
    my_env["LD_LIBRARY_PATH"] = CPLEXPATH
    out_file =  OPTPath + f'out_{t}.dat'
    with open(out_file,'w') as output_f:
        subprocess.check_call([CPLEXPATH+"oplrun", modfile, datafile], stdout=output_f, env=my_env)
    output_f.close()

    # 3. collect results from file
    flow = defaultdict(float)
    with open(resfile,'r', encoding="utf8") as file:
        for row in file:
            item = row.strip().strip(';').split('=')
            if item[0] == 'flow':
                values = item[1].strip(')]').strip('[(').split(')(')
                for v in values:
                    if len(v) == 0:
                        continue
                    i,j,f = v.split(',')
                    flow[int(i),int(j)] = float(f)
    action = [flow[i,j] for i,j in env.edges]
    return action

In [7]:
scenario = Star2Complete(sd = 10, grid_travel_time = 2, star_demand = 6, complete_demand=0.15, 
                         star_center = [5,6,9,10], alpha = 0.3, ninit = 200, fix_price = True)
env = AMoD(scenario)
demand = sum([env.demand[i,j][t] for i,j in env.demand for t in range(0,60)])
print(f"Tot Demand: {demand} -> Should be equal to 25479 as in notebooks/scenario_v2/demo-mpc-new.ipynb")

Tot Demand: 26801 -> Should be equal to 25479 as in notebooks/scenario_v2/demo-mpc-new.ipynb


In [4]:
# three levels
# scenario = Star2Complete(sd = 10, grid_travel_time = 2, T = 16, star_demand = 7.5, complete_demand=1.5, 
#                         star_center = [9,10,13,14], beta=0.9, alpha = 0.5, ninit = 200, fix_price = True)

scenario = Star2Complete(sd = 10, grid_travel_time = 2, star_demand = 6, complete_demand=0.15, 
                         star_center = [5,6,9,10], alpha = 0.3, ninit = 200, fix_price = True)

env = AMoD(scenario)
mpc = MPC(env, CPLEXPATH, T = 12)
opt_rew = []
obs = env.reset()
done = False
served = 0
rebcost = 0
opcost = 0
revenue = 0
paxFlow = dict()
rebFlow = dict()
desiredAcc = dict()
rebAction = dict()
while(not done):
    t = env.time
    res_path = 'tri-level/'
    desiredAcc[t], paxFlow[t], rebFlow[t] = mpc.tri_level() 
    obs, reward1, done, info = env.pax_step(CPLEXPATH = CPLEXPATH, PATH = res_path)
    
    rebAction[t] = solveRebFlow(env,'reb_'+res_path,desiredAcc[t],CPLEXPATH)
    obs, reward2, done, info = env.reb_step(rebAction[t])
    opt_rew.append(reward1+reward2) 
    served += info['served_demand']
    rebcost += info['rebalancing_cost']
    opcost += info['operating_cost']
    revenue += info['revenue']
print(f'Downgraded MPC (three levels): Reward {sum(opt_rew):.2f}, Revenue {revenue:.2f},Served demand {served:.2f}, Rebalancing Cost {rebcost:.2f}, Operational Cost {opcost:.2f}')
demand = sum([env.demand[i,j][t] for i,j in env.demand for t in range(0,60)])
print(demand, served/demand)

ValueError: could not convert string to float: '-1.563194019e-13h\x1e3'